In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [2]:
from matplotlib import pyplot as plt
import sys
import matplotlib.animation as animation
import scipy.sparse as sparse
import scipy.io as sio
import scipy.stats as stats
import scipy.spatial.distance as dist
import numpy as np
import h5py
from tqdm import tqdm
import pandas as pd
import imageio
import os 

#import tables
#import deepdish as dd

sys.path.append('/Users/johnmarshall/Documents/Analysis/PythonAnalysisScripts/post_cmfe_analysis')
import python_utils_jjm as jjm_utils

In [11]:
dir_files = [f_name for f_name in os.listdir('/Volumes/My_Passport/cnmfe_analysis_files/batch_output_files/0102021_files/') if 'out.mat' in f_name]

In [12]:
results = sio.loadmat('/Volumes/My_Passport/cnmfe_analysis_files/batch_output_files/0102021_files/' + '31-Jan_00_12_24_out.mat')

In [13]:
#results['P'][0][0][13]

In [15]:
## return a dictionary matching the batch output file to the information from quest directories 
file_info = {}
files_with_errors = []
for file in dir_files:
    results = sio.loadmat('/Volumes/My_Passport/cnmfe_analysis_files/batch_output_files/0102021_files/' + file)
    try:
        file_ids = [results['P'][0][0][13][0][0][0].split('/')[y] for y in range(5, 7)]
        file_ids.append([results['P'][0][0][13][batch_folder][0][0].split('/')[7] for batch_folder in range(len(results['P'][0][0][13]))])
        file_ids.append([results['P'][0][0][13][batch_folder][0][0].split('/')[8] for batch_folder in range(len(results['P'][0][0][13]))])
        file_info[file] = file_ids
        print(file)
        print('ok')
    except IndexError:
        print('error with')
        print(file)
        files_with_errors.append(file)
        pass
    

29-Jan_12_35_06_out.mat
ok
30-Jan_14_00_19_out.mat
ok
30-Jan_16_54_31_out.mat
ok
31-Jan_00_12_24_out.mat
ok
31-Jan_01_43_49_out.mat
ok


In [16]:
file_info['31-Jan_00_12_24_out.mat'][0]

'GRIN041'

In [18]:
file_info['31-Jan_00_12_24_out.mat'][1]

'1_25_2021'

In [19]:
# display list of file with associated animal numbers
mouse_file_dict = {}
for cnmfe_file in list(file_info.keys()):
    mouse_file_dict[cnmfe_file] = [file_info[cnmfe_file][0], file_info[cnmfe_file][1]]
    

In [20]:
mouse_file_dict

{'29-Jan_12_35_06_out.mat': ['GRIN039', '12_21_2020'],
 '30-Jan_14_00_19_out.mat': ['GRIN038', '12_10_2020'],
 '30-Jan_16_54_31_out.mat': ['GRIN038', '12_11_2020'],
 '31-Jan_00_12_24_out.mat': ['GRIN041', '1_25_2021'],
 '31-Jan_01_43_49_out.mat': ['GRIN038', '12_9_2020']}

In [29]:
mouse_file_df = pd.DataFrame(mouse_file_dict)
mouse_file_df.to_csv('/volumes/My_Passport/dlc_analysis/behavcamvideos/cnmfe_file_key.csv')

In [18]:
behav_cam_path = '/volumes/My_Passport/dlc_analysis/behavcamvideos/'
#for mat_file in list(mouse_file_dict.keys()):
    

In [22]:
file = '10-Jan_21_07_47_out.mat'

animal = mouse_file_dict[file][0]
session = mouse_file_dict[file][1]

path_to_tracking = behav_cam_path + animal +'_' + session + '/' + animal + '_' + session + '_dlc_tracking_foranalysis_04142020.csv'
print(path_to_tracking )

/volumes/My_Passport/dlc_analysis/behavcamvideos/GRIN026_H16_M35_S34/GRIN026_H16_M35_S34_dlc_tracking_foranalysis_04142020.csv


In [19]:
session_name=3
dlc_tracking = str(behav_cam_path+mouse_file_dict[list(mouse_file_dict.keys())[session_name]][0]+'_'+mouse_file_dict[list(mouse_file_dict.keys())[session_name]][1]+'/'

'/volumes/My_Passport/dlc_analysis/behavcamvideos/GRIN026_H16_M35_S34'

In [9]:
files_with_errors

['13-Apr_15_30_16_out.mat',
 '26-Mar_22_01_01_out.mat',
 '29-Mar_11_29_08_out.mat',
 '29-Mar_20_07_14_out.mat',
 '30-Mar_11_06_01_out.mat',
 '31-Mar_09_49_26_out.mat']

In [ ]:
results = sio.loadmat('/Volumes/My_Passport/cnmfe_analysis_files/batch_output_files/' + '08-Apr_16_03_01_out.mat')